In [1]:
# %pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs jieba 

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Connect to the TCM database (created by TCM triple entities) in Neo4j

In [6]:
from langchain_community.graphs import Neo4jGraph
import os

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "88888888"

graph = Neo4jGraph()

In [ ]:
# Visualize the TCM data by specific 病症

In [7]:
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget

# create a neo4j session to run queries
driver = GraphDatabase.driver(uri = os.environ["NEO4J_URI"], auth = (os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]))

# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (n)-[r]-(b) Where n.name='肛门坠胀' RETURN n,r,b LIMIT 10"

def showGraph(cypher: str = default_cypher):
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'name'
    #display(widget)
    return widget

showGraph()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [ ]:
# List out the labels that the graph currently holds

In [4]:
label_list = set()

# Execute a Cypher query to retrieve the available node labels
with driver.session() as session:
    result = session.run("MATCH (n) RETURN DISTINCT labels(n) AS labels")

    # Print the available labels
    for record in result:
        labels = record['labels']
        label_list.update(labels)

print(label_list)

{'chemical_MM', '治法', '贮藏', '用量', '毒性', '归经', '注意', '功能', '病症', 'symmap_chemical', '证候', '中药', '药性', '药味', 'TS_MS', '部位', '用法'}


In [ ]:
# Pick the most appropriate one that we gonna start with and check it's relative properties

In [5]:
# Define the chosen node label
node_label = "病症"  # Replace with the chosen node label

# Execute a Cypher query to retrieve the properties of nodes with the chosen label
with driver.session() as session:
    result = session.run(f"MATCH (n:{node_label}) UNWIND keys(n) AS property RETURN DISTINCT property")

    # Extract the property names
    text_node_properties = [record['property'] for record in result]

    # Print the property names
    for property_name in text_node_properties:
        print(property_name)

name
embedding


In [8]:
API_SECRET_KEY = "sk-KEY";

os.environ["OPENAI_API_KEY"] = API_SECRET_KEY

In [ ]:
# Step 1: Unstructured data retriever

In [ ]:
# Use Neo4jVector.from_existing_graph method to generate a vector index which can find relevant 病症 by their name.
# Calculate the text embedding values for each 病症 based on their name.

In [ ]:
# Hybrid Retrieval: combines vector similarity search and keyword search with graph retrieval
# This retriever employs keyword and vector searches to search through unstructured text data

In [9]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

# Initialize keyword and vector retrieval from existing graph
# vector_index = Neo4jVector.from_existing_graph(
#     embedding=OpenAIEmbeddings(),
#     search_type="hybrid",
#     node_label="病症",
#     text_node_properties=["name"],
#     embedding_node_property="embedding",
# )

# load the hybrid search from existing indexes (if the keyword and vector retrieval has been initialized)
index_name = "vector" 
keyword_index_name = "keyword"

vector_index = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="病症",
    text_node_properties=["name"],
    embedding_node_property="embedding",
    index_name=index_name,
    keyword_index_name=keyword_index_name,
)

In [ ]:
# Find nodes that are similar to a given vector using a similarity metric (kNN).

In [22]:
result_list = vector_index.similarity_search("肛门痛是怎么回事?")
unstructured_data = [doc.page_content.replace('\nname: ', '') for doc in result_list]
print(unstructured_data)

['肛门疼痛', '肛门肿痛', '肛门坠胀疼痛', '肛门疼痛剧烈']


In [ ]:
# Step2: Graph retriever

In [ ]:
# Graph retriever: use a full-text index to identify relevant nodes and then return their direct neighborhood.

In [ ]:
# We starts by identifying relevant entities in the input. 
# For simplicity, we instruct the LLM to identify 中药, 证候, 病症.

In [29]:
# 使用 Cypher 查询创建一个名为 "entity" 的全文索引，该索引将在每个节点的 name 属性上进行索引。
graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (n:中药 | 部位 | 归经 | 证候 | 用法 | 病症 | 治法 | 贮藏 | 功能 | 毒性 | 注意 | 药性 | 药味 | 用量) ON EACH [n.name]")

[]

In [11]:
cypher_query = "SHOW INDEXES"

with driver.session() as session:
    result = session.run(cypher_query)
    # Check if the "entity" index exists
    for record in result:
        index_name = record["name"]
        print(index_name)

entity
index_343aff4e
index_f7700477
keyword
vector


In [12]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

# 从文本中提取相关实体
class Entities(BaseModel):
    """识别与中医(中国传统医学)、中药相关的实体信息."""

    names: List[str] = Field(
        ...,
        description="文本中出现的所有中药、证候和病症实体",
    )

# https://python.langchain.com/docs/modules/model_io/chat/structured_output/
structured_llm = llm.with_structured_output(Entities)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "您正在从文本中提取中药、证候和病症实体。",
        ),
        (
            "human",
            "请使用给定的格式从以下内容中提取信息 "
            "输入: {question}",
        ),
    ]
)

# 使用在prompt变量中定义的模板设置prompt
# 配置llm以生成与Entities数据模型匹配的结构化响应，使得模型能够从提供的文本中提取与中医、中药、症状和证候相关的实体信息
# entity_chain以Entities对象的结构化输出形式回复您，该输出基于提供的模板和配置的语言模型。
entity_chain = prompt | structured_llm

/Users/infung/anaconda3/envs/KG/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [14]:
entity_chain.invoke({"question": "肛门痛是怎么回事?"}).names

['肛门痛']

In [ ]:
# Use the full-text index to map the extracted entities to the knowledge graph

In [15]:
import jieba # 中文分词

# 使用jieba进行中文分词
# 再使用Lucene的AND搜索语法进行全文搜索
# "肛门痛" -> "肛门 AND 痛"
def generate_full_text_query(input: str) -> str:
    """
    为给定的输入字符串生成全文搜索查询（full-text search query）。

    该函数构建一个适用于全文搜索查询（full-text search query）的字符串。
    它通过将输入字符串分拆成词语，并在每个词语后附加一个相似度阈值（大约1个改变的字符）来处理输入字符串。
    然后使用AND操作符将它们组合起来。可用于将用户问题中的实体映射到数据库值，并允许一定程度的错误用词。
    """
    full_text_query = ""
    words = jieba.lcut(input)
    for word in words[:-1]:
        full_text_query += f" {word} AND"
    full_text_query += f" {words[-1]}"
    return full_text_query.strip()

print(generate_full_text_query("肛门痛"))

肛门 AND 痛


In [16]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    收集在问题中提到的实体的邻居信息
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query)
            YIELD node
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              WHERE NOT type(r) IN ['symmap_chemical', 'chemical_MM', 'TS_MS']
              RETURN node.name + ' - ' + type(r) + ' -> ' + neighbor.name AS output
              UNION
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              WHERE NOT type(r) IN ['symmap_chemical', 'chemical_MM', 'TS_MS']
              RETURN neighbor.name + ' - ' + type(r) + ' -> ' +  node.name AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [25]:
print(structured_retriever("肛门痛是怎么回事?"))

肛门疼痛 - 中药 -> 紫苏子
肛门疼痛 - 中药 -> 蕤仁
肛门疼痛 - 证候 -> 气血亏虚证
肛门疼痛 - 中药 -> 三七
肛门疼痛 - 中药 -> 板蓝根
肛门疼痛 - 中药 -> 安息香
肛门肿痛 - 证候 -> 溃脓期
肛门灼热疼痛 - 证候 -> 湿热下注证
肛门坠胀疼痛 - 证候 -> 血热瘀阻证
肛门坠胀疼痛 - 证候 -> 湿热下注证
肛门坠胀疼痛 - 证候 -> 湿热壅滞证
肛门疼痛剧烈 - 证候 -> 气血瘀阻证
肛门红肿灼热疼痛 - 证候 -> 肛门热毒证


In [ ]:
# Step3: Final retriever

In [ ]:
# Combine the unstructured and graph retriever to create the final context that will be passed to an LLM.

In [33]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    result_list = vector_index.similarity_search(question)
    unstructured_data = [doc.page_content.replace('\nname: ', '') for doc in result_list]
    final_data = f"""结构化数据:
{structured_data}
非结构化数据:
{"#病症 ". join(unstructured_data)}
    """
    return final_data

In [38]:
from langchain_core.prompts.prompt import PromptTemplate
from typing import Tuple, Optional
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Condense a chat history and follow-up question into a standalone question
_template = """给定以下对话和后续问题，请将后续问题改写为一个独立的问题，用其原始语言表达。
对话历史:
{chat_history}
后续输入: {question}
独立问题:"""  

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [ ]:
# a prompt that leverages the context provided by the integrated hybrid retriever to produce the response
# completing the implementation of the RAG chain.

In [39]:
template = """仅基于以下上下文回答问题：
{context}

问题: {question}
使用自然语言（中文），并简明扼要地回答。
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [40]:
chain.invoke({"question": "肛门痛是怎么回事?"})

Search query: 肛门痛是怎么回事?


'肛门痛可能是由于气血亏虚、湿热下注、血热瘀阻等不同证候引起的。针对不同的病症，可以使用紫苏子、蕤仁、三七、板蓝根、安息香等中药进行治疗。'

In [41]:
chain.invoke({"question": "我最近便秘，怎么办？"})

Search query: 我最近便秘，怎么办？


'您最近便秘可能是由于身体内部的阴虚、阳虚、气滞等原因导致的。建议您可以尝试调整饮食，多食用易消化的食物，如山药、黑豆等，同时可以考虑服用一些中药如没药、石斛等进行调理。如果情况持续或加重，请及时就医。'

In [ ]:
# done!